In [34]:
# Install dependencies
!pip install -q unsloth transformers qwen-vl-utils

In [35]:
# 1. Load libraries
import pandas as pd
import numpy as np
import torch
from PIL import Image
from unsloth import FastVisionModel
from transformers import AutoProcessor
from qwen_vl_utils import process_vision_info
from sklearn. metrics import f1_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 2. Load Qwen2. 5-VL-7B-Instruct (4-bit NF4 with Unsloth)
model, tokenizer = FastVisionModel.from_pretrained(
    model_name="Qwen/Qwen2.5-VL-7B-Instruct",
    load_in_4bit = False,
    load_in_8bit=True,
    max_seq_length=2048,
    device_map="auto",
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-7B-Instruct")

print("✓ Model loaded successfully!")

==((====))==  Unsloth 2025.11.4: Fast Qwen2_5_Vl patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

In [ ]:
# Updated binary-classification prompt for meme decoding
prompt = """
You are an expert in analyzing memes and determining whether they contain political content.

Classify this meme into ONE of the following two categories:

1. **Political** – The meme targets, references, or implies anything related to:
   - politicians or political leaders
   - political parties or ideologies
   - government policies, elections, movements, activism
   - political events, national issues, or public administration
   - political satire, criticism, propaganda, or commentary

2. **NonPolitical** – The meme does NOT contain political intent.  
   This includes humor, general social topics, personal jokes, pop culture, relationships, gender jokes, daily life, or any content unrelated to politics.

Important guidelines:
- Consider both explicit and implicit political meaning.
- Detect political symbolism, colors, slogans, and indirect references.
- Ignore OCR noise; focus on the dominant meaning.
- Sarcasm or indirect jokes still count as Political **if** they relate to politics.

Respond with ONLY ONE word: **Political** or **NonPolitical**.
"""

print("Prompt created")

In [ ]:
# 3. Define inference function (CORRECTED)
def predict_image(image_path, prompt_text):
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": prompt_text}
            ]
        }
    ]
    
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    image = Image.open(image_path). convert("RGB")
    
    # FIXED: process_vision_info returns only 2 values
    image_inputs, video_inputs = process_vision_info(messages)
    
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt"
    ). to(model.device)
    
    outputs = model. generate(
        **inputs,
        max_new_tokens=50,
        temperature=0.1
    )
    
    result = processor.batch_decode(outputs, skip_special_tokens=True)[0]
    
    # Clean output for binary classification
    result = result.strip().split()[-1].strip('.,!? "\'')
    
    # Normalize
    res = result.lower().strip()
    
    # Exact match first
    if res in ["political", "politics"]:
        return "Political"
    elif res in ["nonpolitical", "non-political", "not political", "not-political"]:
        return "NonPolitical"

In [ ]:
# Load test CSV
test_csv_path = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Test.csv"
test_images_dir = "/kaggle/input/poli-meme-decode-cuet-cse-fest/PoliMemeDecode/Test/Image"

df_test = pd.read_csv(test_csv_path)
# df_test = df_test.head(20)
print(f"Total test samples: {len(df_test)}")

In [ ]:
# Run predictions on test set
test_preds = []

for _, row in tqdm(df_test. iterrows(), total=len(df_test), desc="Predicting test images"):
    image_filename = row['Image_name']
    image_path = os.path.join(test_images_dir, image_filename)
    
    if not os.path.exists(image_path):
        print(f"Warning: Image not found - {image_path}")
        test_preds.append("NonPolitical")
        continue
    
    pred = predict_image(image_path, prompt)
    test_preds.append(pred)

print(f"✓ Test predictions complete!  Processed {len(test_preds)} images")

In [ ]:
# Create Test submission file
submission = pd.DataFrame({
    'Image_name': df_test['Image_name'],
    'Target': test_preds
})

submission. to_csv('submission.csv', index=False)
print("✓ Submission file saved to 'submission.csv'")
print("\nFirst 10 predictions:")
print(submission.head(20))